In [6]:
import csv
import operator
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [9]:
df = pd.read_csv('../datasets/land-use-over-the-long-term.csv')
land_use_columns = ['Land use: Built-up area', 'Land use: Grazingland', 'Land use: Cropland']
degree = 3
future_years = np.arange(2024, 2125)
all_predictions = []

for country in df['Entity'].unique():
    country_predictions = {'Entity': country}
    
    for year in future_years:
        country_predictions['Year'] = year
        
        for land_use_type in land_use_columns:
            country_df = df[(df['Entity'] == country)]
            
            X = country_df['Year'].values.reshape(-1, 1)
            y = country_df[land_use_type].values
            
            poly = PolynomialFeatures(degree=degree)
            X_poly = poly.fit_transform(X)
            
            model = LinearRegression()
            model.fit(X_poly, y)
            
            year_poly = poly.transform(np.array([[year]]))
            prediction = model.predict(year_poly)[0]
            
            country_predictions[land_use_type] = prediction
        
        all_predictions.append(country_predictions.copy())

predictions_df = pd.DataFrame(all_predictions)

columns_order = ['Entity', 'Year'] + land_use_columns
predictions_df = predictions_df[columns_order]

csv_file_path = 'land_use_predictions_2024_2124.csv'
predictions_df.to_csv(csv_file_path, index=False)